# Webscraping allrecipes.com
<b>Edie Espejo</b>
<br>
2017-09-15 | 2017-09-07

<b>Link</b>: http://allrecipes.com/recipes/343/bread/quick-bread/fruit-bread/banana-bread/?internalSource=hubcard&referringContentType=search%20results&clickId=cardslot%201

# Install BeautifulSoup

In [2]:
# import pip
# pip.main(["install", "bs4"])

  Running setup.py bdist_wheel for bs4: started
  Running setup.py bdist_wheel for bs4: finished with status 'done'
  Stored in directory: C:\Users\ehze\AppData\Local\pip\Cache\wheels\84\67\d4\9e09d9d5adede2ee1c7b7e8775ba3fbb04d07c4f946f0e4f11
Successfully built bs4


0

In [12]:
# pip.main(["install", "requests_cache"])

0

In [182]:
# import pip
# pip.main(["install", "time"])

  Could not find a version that satisfies the requirement time (from versions: )
No matching distribution found for time


1

In [ ]:
# import time 
# while True:
#     print("This prints once a minute.")
#     time.sleep(60)   # Delay for 1 minute (60 seconds).

In [1]:
# IMPORTANT LIBRARIES
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import requests_cache

### Don't forget to CACHE!

In [ ]:
# START CACHING
requests_cache.install_cache("banana_cache_money")

# Collect Links
It seems that the links are formatted in this fashion:
http://allrecipes.com/recipes/343/bread/quick-bread/fruit-bread/banana-bread/?internalSource=hubcard&referringContentType=search%20results&clickId=cardslot%201&page=5#5

where we can replace the last numbers to resemble integers from 2 to 9.

In [34]:
# FORMAT LINKS
home = "http://allrecipes.com"
base_link = home + "/recipes/343/bread/quick-bread/fruit-bread/banana-bread/?internalSource=hubcard&referringContentType=search%20results&clickId=cardslot%201&page="

all_links = [base_link+str(integer)+"#"+str(integer) for integer in range(1,10)]

In [35]:
all_links

['http://allrecipes.com/recipes/343/bread/quick-bread/fruit-bread/banana-bread/?internalSource=hubcard&referringContentType=search%20results&clickId=cardslot%201&page=1#1',
 'http://allrecipes.com/recipes/343/bread/quick-bread/fruit-bread/banana-bread/?internalSource=hubcard&referringContentType=search%20results&clickId=cardslot%201&page=2#2',
 'http://allrecipes.com/recipes/343/bread/quick-bread/fruit-bread/banana-bread/?internalSource=hubcard&referringContentType=search%20results&clickId=cardslot%201&page=3#3',
 'http://allrecipes.com/recipes/343/bread/quick-bread/fruit-bread/banana-bread/?internalSource=hubcard&referringContentType=search%20results&clickId=cardslot%201&page=4#4',
 'http://allrecipes.com/recipes/343/bread/quick-bread/fruit-bread/banana-bread/?internalSource=hubcard&referringContentType=search%20results&clickId=cardslot%201&page=5#5',
 'http://allrecipes.com/recipes/343/bread/quick-bread/fruit-bread/banana-bread/?internalSource=hubcard&referringContentType=search%20re

The markup sort of looks like this.

        <h3 class="grid-col__h3 grid-col__h3--recipe-grid">
            Chef John's Banana Bread
                <div class="grid-col__video">
                    <a href="/video/528/chef-johns-banana-bread/" data-internal-referrer-link='hub recipe' data-click-id='cardslot 1'><span class="icon--videoplay-small-white"></span></a>
                </div>
        </h3>

In [50]:
# FUNCTION TO RETRIEVE URLS FROM ONE PAGE
def one_page_recipes(url):
    soup = BeautifulSoup((requests.get(url)).text, "html.parser")
    page_recipes = soup.find_all("article", {"class", "grid-col--fixed-tiles"})
    
    # INITIALIZE LIST
    recipes = []
    for page in page_recipes:
        try:
            temp = page.find_all("a")[0]["href"]
            if temp.startswith("/"):
                recipes.append(temp)
        except:
            continue
                  
    return(recipes)

In [54]:
banana_bread_links = [one_page_recipes(this_link) for this_link in all_links]

In [71]:
banana_bread_links[0]

['/recipe/7158/strawnana-bread/',
 '/recipe/6765/4h-banana-bread/',
 '/recipe/218541/caramel-macchiato-banana-bread/',
 '/recipe/219967/chef-johns-banana-bread/',
 '/recipe/7154/creamy-banana-bread/',
 '/recipe/17252/chocolate-banana-bread/',
 '/recipe/228442/raspberry-banana-bread/',
 '/recipe/16999/grandmas-homemade-banana-bread/',
 '/recipe/141933/banana-zucchini-bread/',
 '/video/4744/banana-chocolate-chip-bread/',
 '/recipe/17652/banana-crumb-muffins/',
 '/recipe/20144/banana-banana-bread/',
 '/recipe/17066/janets-rich-banana-bread/',
 '/recipe/6984/banana-sour-cream-bread/',
 '/video/1065/extreme-banana-nut-bread/',
 '/recipe/67937/extreme-banana-nut-bread-ebnb/',
 '/recipe/15747/best-ever-banana-bread/',
 '/recipe/16952/the-best-banana-bread/',
 '/recipe/6687/banana-bread/',
 '/video/673/banana-banana-bread/',
 '/recipe/6993/cathys-banana-bread/',
 '/video/528/chef-johns-banana-bread/',
 '/recipe/6990/sarahs-banana-bread-muffins/',
 '/recipe/23426/almost-no-fat-banana-bread/',
 

In [72]:
# CONCATENATE LIST AND FORMAT UNIVERSAL LINKS
bblinks = [home+list_item for sublist in banana_bread_links for list_item in sublist]

In [73]:
bblinks[0:5]

['http://allrecipes.com/recipe/7158/strawnana-bread/',
 'http://allrecipes.com/recipe/6765/4h-banana-bread/',
 'http://allrecipes.com/recipe/218541/caramel-macchiato-banana-bread/',
 'http://allrecipes.com/recipe/219967/chef-johns-banana-bread/',
 'http://allrecipes.com/recipe/7154/creamy-banana-bread/']

In [74]:
len(bblinks)

181

We need to remove the videos.

In [164]:
bblinks_novids = [link for link in bblinks if "video" not in link]

In [165]:
len(bblinks_novids)

177

Now the links have been collected, let's continue on to check out how to read one of the pages.

# Gathering Page Information
We're going to inspect one of the pages, then create a function that can be generalized to retrieve:
<br>
(1) Recipe Name
<br>
(2) Star Rating & Review Count
<br>
(4) Ingredients
<br>
(5) Calories

In [75]:
bblinks[0]

'http://allrecipes.com/recipe/7158/strawnana-bread/'

### Recipe Name
The markup for the recipe name looks like this. 

In [78]:
# <section class="recipe-summary clearfix">
#     <meta property="og:rating" content="4.41201734542847" />
#     <meta property="og:rating_scale" content="5" />
#     <div id="karma-lazy-sponsorLogo" class="recipe-summary__logo"></div>
#     <h1 class="recipe-summary__h1" itemprop="name">Strawnana Bread</h1>
#     <!--rating stars-->
#     <div class="recipe-summary__stars">
#         <div class="rating-stars" data-scroll-to-anchor="reviews" data-ratingstars= 4.41201734542847 >

In [77]:
#         <span itemprop="aggregateRating" itemscope itemtype="http://schema.org/AggregateRating" class="aggregate-rating">
#             <meta itemprop="ratingValue" content="4.41">
#             <meta itemprop="reviewCount" content="182">

In [82]:
soup = BeautifulSoup((requests.get(bblinks[0])).text, "html.parser")
clearfix = soup.find_all("section", {"class", "recipe-summary clearfix"})[0]

In [83]:
clearfix

<section class="recipe-summary clearfix">
<meta content="4.41201734542847" property="og:rating"/>
<meta content="5" property="og:rating_scale"/>
<div class="recipe-summary__logo" id="karma-lazy-sponsorLogo"></div>
<h1 class="recipe-summary__h1" itemprop="name">Strawnana Bread</h1>
<!--rating stars-->
<div class="recipe-summary__stars">
<div class="rating-stars" data-ratingstars="4.41201734542847" data-scroll-to-anchor="reviews">
<img height="20" src="http://images.media-allrecipes.com/ar-images/icons/rating-stars/full-star-2015.svg" width="20">
<img height="20" src="http://images.media-allrecipes.com/ar-images/icons/rating-stars/full-star-2015.svg" width="20">
<img height="20" src="http://images.media-allrecipes.com/ar-images/icons/rating-stars/full-star-2015.svg" width="20">
<img height="20" src="http://images.media-allrecipes.com/ar-images/icons/rating-stars/full-star-2015.svg" width="20">
<img height="20" src="http://images.media-allrecipes.com/ar-images/icons/rating-stars/half-star

In [95]:
clearfix.find("h1").contents[0]

'Strawnana Bread'

### Star Rating & Review Count

In [106]:
clearfix.find("div", {"class": "recipe-summary__stars"}).find("div", {"class", "rating-stars"})["data-ratingstars"]

'4.41201734542847'

In [112]:
metas = clearfix.find("div", {"class": "recipe-summary__stars"}).find("span", {"class", "aggregate-rating"}).find_all("meta")

In [113]:
metas

[<meta content="4.41" itemprop="ratingValue"/>,
 <meta content="182" itemprop="reviewCount"/>]

In [114]:
[h["content"] for h in metas]

['4.41', '182']

### Ingredients

In [ ]:
#   <ul class="checklist dropdownwrapper list-ingredients-1" ng-hide="reloaded" id="lst_ingredients_1">
#                 <li class="checkList__line">
#                     <label ng-class="{true: 'checkList__item'}[true]">
#                         <input data-id="16317" name="ingredientCheckbox" data-role="none" type="checkbox" value="N" ng-click="saveIngredient($event,16317)" />
#                         
#                     </label>
#                         <div data-ng-repeat='deal in deals["16317"]'>
#                             <div class="offer-container" data-ng-class='{"hide-deals":!deals["16317"] || !localOffersService.localOffersUserEnabled()}' ng-cloak>
#                                 <div class="left">
#                                     <p>
#                                         <span class="offer-name" data-ng-bind-html='deal.description'></span>
#                                         <br/>
#                                         <span ng-show='deal.isBIU' data-ng-bind-html='deal.price'></span>
#                                         <span ng-show='!deal.isBIU' class='unit-cost' data-ng-bind-html='deal.priceWithExpiration'></span>
#                                         <br/>
#                                         <a data-no-follow-if-external ng-href='{{deal.actionUrl}}' target="_blank" data-ng-click='fireClickPixels(deal.clickPixels)' data-ng-bind-html='deal.clickThroughText'></a>
#                                         <br/>
#                                         <span class="advertisement" ng-show='deal.isBIU'>ADVERTISEMENT</span>
#                                     </p>
#                                     <div class="tracking-element" load-dom-script script="deal.moatUrl"></div>
#                                     <img class="tracking-element" data-ng-repeat="trackingUrl in deal.trackingPixels track by $index" data-ng-src="{{trackingUrl}}" />
#                                     <img class="tracking-element" data-ng-repeat="clickUrl in listClickPixels track by $index" data-ng-src="{{clickUrl}}" />
#                                 </div>
#                                 <div class="right">
#                                     <img data-ng-src='{{deal.imageUrl}}' class="offer-photo" />
#                                 </div>
#                             </div>
#                         </div>
#                 </li>
#                 <li class="checkList__line">
#                     <label ng-class="{true: 'checkList__item'}[true]">
#                         <input data-id="1526" name="ingredientCheckbox" data-role="none" type="checkbox" value="N" ng-click="saveIngredient($event,1526)" />
#                         <span class="recipe-ingred_txt added" data-id="1526" data-nameid="1526" itemprop="ingredients">2 cups white sugar</span>
#                     </label>
#                         <div data-ng-repeat='deal in deals["1526"]'>
#                             <div class="offer-container" data-ng-class='{"hide-deals":!deals["1526"] || !localOffersService.localOffersUserEnabled()}' ng-cloak>
#                                 <div class="left">
#                                     <p>
#                                         <span class="offer-name" data-ng-bind-html='deal.description'></span>
#                                         <br/>
#                                         <span ng-show='deal.isBIU' data-ng-bind-html='deal.price'></span>
#                                         <span ng-show='!deal.isBIU' class='unit-cost' data-ng-bind-html='deal.priceWithExpiration'></span>
#                                         <br/>
#                                         <a data-no-follow-if-external ng-href='{{deal.actionUrl}}' target="_blank" data-ng-click='fireClickPixels(deal.clickPixels)' data-ng-bind-html='deal.clickThroughText'></a>
#                                         <br/>
#                                         <span class="advertisement" ng-show='deal.isBIU'>ADVERTISEMENT</span>
#                                     </p>
#                                     <div class="tracking-element" load-dom-script script="deal.moatUrl"></div>
#                                     <img class="tracking-element" data-ng-repeat="trackingUrl in deal.trackingPixels track by $index" data-ng-src="{{trackingUrl}}" />
#                                     <img class="tracking-element" data-ng-repeat="clickUrl in listClickPixels track by $index" data-ng-src="{{clickUrl}}" />
#                                 </div>
#                                 <div class="right">
#                                     <img data-ng-src='{{deal.imageUrl}}' class="offer-photo" />
#                                 </div>
#                             </div>
#                         </div>
#                 </li>

In [ ]:
# <span class="recipe-ingred_txt added" data-id="16317" data-nameid="16317" itemprop="ingredients">4 eggs, lightly beaten</span>

In [121]:
ingredients = soup.find_all("span", {"class", "recipe-ingred_txt added"})

In [122]:
len(ingredients)

12

In [126]:
[item.contents[0] for item in ingredients]

['4 eggs, lightly beaten',
 '2 cups white sugar',
 '1 1/2 cups strawberries, mashed',
 '1 cup mashed bananas',
 '1 tablespoon orange zest',
 '1 cup vegetable oil',
 '3 cups all-purpose flour',
 '1 teaspoon baking soda',
 '1 1/2 teaspoons ground cinnamon',
 '1/2 teaspoon salt',
 '1/4 teaspoon ground nutmeg',
 '1 cup chopped walnuts']

### Calories

In [127]:
# <span class="calorie-count" ng-class="{'active': nutritionSection_showing}"><span>258</span> <span class="calorie-count__desc" ng-class="{'active': nutritionSection_showing}"> cals</span></span>

In [143]:
soup.find_all("span", {"class", "calorie-count"})[0].find("span").contents[0]

'258'

### Function To Retrieve All
Now, what remains is the task to make a function to wrangle these data from a generic page!

In [ ]:
# def read_a_page(url):
#     # CONVERT PAGE
#     soup = BeautifulSoup((requests.get(url)).text, "html.parser")
    
#     # RECIPE TITLE
#     try:
#         clearfix = soup.find_all("section", {"class", "recipe-summary clearfix"})[0]
#         title = clearfix.find("h1").contents[0]
#     except:
#         print(url)

#     # RATINGS
#     try:
#         longRating = clearfix.find("div", {"class": "recipe-summary__stars"}).find("div", {"class", "rating-stars"})["data-ratingstars"]
#         metas = clearfix.find("div", {"class": "recipe-summary__stars"}).find("span", {"class", "aggregate-rating"}).find_all("meta")
#         metas_content = [h["content"] for h in metas]
#         ratingValue, reviewCount = metas_content[0], metas_content[1]
#     except:
#         print(url)
    
#     # INGREDIENTS
#     try:
#         ingredients = soup.find_all("span", {"class", "recipe-ingred_txt added"})
#         recipe = [item.contents[0] for item in ingredients]
#     except:
#         print(url)
                         
#     # CALORIES
#     try:
#         calories = soup.find_all("span", {"class", "calorie-count"})[0].find("span").contents[0]
#     except:
#         print(url)
    
#     try:
#         return([title, longRating, ratingValue, reviewCount, recipe, calories])
#     except:
#         return(0)

In [178]:
def read_a_page(url):
    # CONVERT PAGE
    soup = BeautifulSoup((requests.get(url)).text, "html.parser")
    
    # RECIPE TITLE
    try:
        clearfix = soup.find_all("section", {"class", "recipe-summary clearfix"})[0]
        title = clearfix.find("h1").contents[0]
    except:
        title = np.NaN

    # RATINGS
    longRating = clearfix.find("div", {"class": "recipe-summary__stars"}).find("div", {"class", "rating-stars"})["data-ratingstars"]
    metas = clearfix.find("div", {"class": "recipe-summary__stars"}).find("span", {"class", "aggregate-rating"}).find_all("meta")
    metas_content = [h["content"] for h in metas]
    ratingValue, reviewCount = metas_content[0], metas_content[1]
    
    # INGREDIENTS
    ingredients = soup.find_all("span", {"class", "recipe-ingred_txt added"})
    recipe = [item.contents[0] for item in ingredients]
                         
    # CALORIES
    try:
        calories = soup.find_all("span", {"class", "calorie-count"})[0].find("span").contents[0]
    except:
        calories = np.NaN
    
    return([url, title, longRating, ratingValue, reviewCount, recipe, calories])                     
    

In [179]:
table = pd.DataFrame([read_a_page(this_link) for this_link in bblinks_novids])

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [162]:
i = 0
for this_link in bblinks:
    print(i)
    print(read_a_page(this_link))
    i = i + 1

0
['Strawnana Bread', '4.41201734542847', '4.41', '182', ['4 eggs, lightly beaten', '2 cups white sugar', '1 1/2 cups strawberries, mashed', '1 cup mashed bananas', '1 tablespoon orange zest', '1 cup vegetable oil', '3 cups all-purpose flour', '1 teaspoon baking soda', '1 1/2 teaspoons ground cinnamon', '1/2 teaspoon salt', '1/4 teaspoon ground nutmeg', '1 cup chopped walnuts'], '258']
1
['4H Banana Bread', '4.67133522033691', '4.67', '881', ['2 cups all-purpose flour', '1/2 teaspoon baking soda', '1 cup white sugar', '1 egg', '5 tablespoons milk', '1 teaspoon baking powder', '1/2 teaspoon salt', '1/2 cup margarine', '1 cup mashed bananas', '1/2 cup chopped walnuts (optional)'], '265']
2
['Caramel Macchiato Banana Bread', '4.18181800842285', '4.18', '57', ['cooking spray', '2 cups all-purpose flour', '1 teaspoon baking soda', '1 teaspoon baking powder', '1/2 teaspoon ground cinnamon', '1/8 teaspoon salt', '2 large very ripe bananas', '1/2 cup caramel macchiato flavored liquid coffee cr

['Whole Wheat Banana Nut Bread', '4.6054630279541', '4.61', '570', ['1/3 cup vegetable oil', '1/2 cup honey', '1 teaspoon vanilla extract', '2 eggs', '1 cup mashed bananas', '1 3/4 cups whole wheat flour', '1/2 teaspoon salt', '1 teaspoon baking soda', '1/4 cup hot water', '1/2 cup chopped walnuts'], '218']
28
['Lighter Banana Muffins', '4.28030300140381', '4.28', '468', ['2 eggs, beaten', '3 very ripe bananas, mashed', '2 cups all-purpose flour', '1 teaspoon salt', '3/4 cup white sugar', '1 teaspoon baking soda'], '195']
29
['Banana Chocolate Chip Bread', '4.75257730484009', '4.75', '388', ['2 cups all-purpose flour', '1 teaspoon baking powder', '1 teaspoon baking soda', '1 teaspoon salt', '3 ripe bananas, mashed', '1 tablespoon milk', '1 teaspoon ground cinnamon, or to taste', '1/2 cup butter, softened', '1 cup white sugar', '2 eggs', '1 cup semisweet chocolate chips'], '378']
30
["Chef John's Banana Bread", '4.69409275054932', '4.69', '380', ['cooking spray', '2 cups all-purpose flo

KeyboardInterrupt: 

In [177]:
bblinks_novids

['http://allrecipes.com/recipe/7158/strawnana-bread/',
 'http://allrecipes.com/recipe/6765/4h-banana-bread/',
 'http://allrecipes.com/recipe/218541/caramel-macchiato-banana-bread/',
 'http://allrecipes.com/recipe/219967/chef-johns-banana-bread/',
 'http://allrecipes.com/recipe/7154/creamy-banana-bread/',
 'http://allrecipes.com/recipe/17252/chocolate-banana-bread/',
 'http://allrecipes.com/recipe/228442/raspberry-banana-bread/',
 'http://allrecipes.com/recipe/16999/grandmas-homemade-banana-bread/',
 'http://allrecipes.com/recipe/141933/banana-zucchini-bread/',
 'http://allrecipes.com/recipe/17652/banana-crumb-muffins/',
 'http://allrecipes.com/recipe/20144/banana-banana-bread/',
 'http://allrecipes.com/recipe/17066/janets-rich-banana-bread/',
 'http://allrecipes.com/recipe/6984/banana-sour-cream-bread/',
 'http://allrecipes.com/recipe/67937/extreme-banana-nut-bread-ebnb/',
 'http://allrecipes.com/recipe/15747/best-ever-banana-bread/',
 'http://allrecipes.com/recipe/16952/the-best-banan

In [183]:
table

NameError: name 'table' is not defined

I got kicked off at home.